In [1]:
import requests
import pandas as pd
import time
pd.set_option('display.max_columns', None)


In [2]:
institution_ror = catalog.load('params:openalex_fetch_options.institution_ror')
#env = catalog.load('params:fetch_options.env')
env = 'dev'

[12/19/24 11:43:40] INFO     Loading data from params:openalex_fetch_options.institution_ror    ]8;id=29560;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=466752;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

In [3]:
env

'dev'

In [4]:
def fetch_work_openalex(institution_ror, env):
    cursor = '*'
    base_url = 'https://api.openalex.org/works?filter=institutions.ror:{}&cursor={}&per-page=200'
    iteration_limit = 5
    iteration_count = 0

    url = base_url.format(institution_ror, cursor)
    api_response = requests.get(url).json()

    print(f'Iteration count: {iteration_count}')
    print(f'GET {url}')

    # creo dataframe con las columnas del primer resultado 
    df = pd.DataFrame.from_dict(api_response['results'])

    # update cursor
    cursor = api_response['meta']['next_cursor']
    url = base_url.format(institution_ror, cursor)

    while cursor:

        if env == 'dev' and iteration_count >= iteration_limit:
            break

        # request api with updated cursor
        url = base_url.format(institution_ror, cursor)

        iteration_count += 1
        print(f'Iteration count: {iteration_count}')
        print(f'GET {url}')

        # Sleep para evitar exceder el límite de solicitudes por segundo
        time.sleep(1)

        api_response = requests.get(url).json()

        df_tmp = pd.DataFrame.from_dict(api_response['results'])       

        if "abstract_inverted_index" in df_tmp.columns:
            df_tmp.drop(columns=["abstract_inverted_index"], inplace=True)


        df = pd.concat([df, df_tmp])

        # update cursor
        cursor = api_response['meta']['next_cursor']

    return df, df.head(1000)

In [5]:
df, df_dev = fetch_work_openalex(institution_ror, env)

Iteration count: 0
GET https://api.openalex.org/works?filter=institutions.ror:https://ror.org/02s7sax82&cursor=*&per-page=200
Iteration count: 1
GET https://api.openalex.org/works?filter=institutions.ror:https://ror.org/02s7sax82&cursor=Ils5Ni4wLCA1NiwgJ2h0dHBzOi8vb3BlbmFsZXgub3JnL1cyMDE1MDYxODQ1J10i&per-page=200


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 df, df_dev = fetch_work_openalex(institution_ror, env)                                       │
│   2                                                                                              │
│                                                                                                  │
│ in fetch_work_openalex:33                                                                        │
│                                                                                                  │
│   30 │   │   print(f'GET {url}')                                                                 │
│   31 │   │                                                                                       │
│   32 │   │   # Sleep para evitar exceder el límite de solicitudes por segundo                    │
│ ❱ 33 │   │   time.sleep(1)                                                                       │
│   34 │   │                                                                                       │
│   35 │   │   api_response = requests.get(url).json()                                             │
│   36                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'time' is not defined

In [6]:
df.head(5)

,id,doi,title,display_name,publication_year,publication_date,ids,language,primary_location,type,type_crossref,indexed_in,open_access,authorships,institution_assertions,countries_distinct_count,institutions_distinct_count,corresponding_author_ids,corresponding_institution_ids,apc_list,apc_paid,fwci,has_fulltext,fulltext_origin,cited_by_count,citation_normalized_percentile,cited_by_percentile_year,biblio,is_retracted,is_paratext,primary_topic,topics,keywords,concepts,mesh,locations_count,locations,best_oa_location,sustainable_development_goals,grants,datasets,versions,referenced_works_count,referenced_works,related_works,abstract_inverted_index,cited_by_api_url,counts_by_year,updated_date,created_date
0,https://openalex.org/W2089852843,https://doi.org/10.1016/s0378-4371(98)00437-3,The role of constraints within generalized non...,The role of constraints within generalized non...,1998,1998-12-01,{'openalex': 'https://openalex.org/W2089852843...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,journal-article,[crossref],"{'is_oa': False, 'oa_status': 'closed', 'oa_ur...","[{'author_position': 'first', 'author': {'id':...",[],3,6,[https://openalex.org/A5051451815],[https://openalex.org/I4210125245],"{'value': 3320, 'currency': 'USD', 'value_usd'...",None,30.222,True,ngrams,1335,"{'value': 0.999702, 'is_in_top_1_percent': Tru...","{'min': 99, 'max': 100}","{'volume': '261', 'issue': '3-4', 'first_page'...",False,False,"{'id': 'https://openalex.org/T12261', 'display...","[{'id': 'https://openalex.org/T12261', 'displa...",[{'id': 'https://openalex.org/keywords/hamilto...,"[{'id': 'https://openalex.org/C40934718', 'wik...",[],1,"[{'is_oa': False, 'landing_page_url': 'https:/...",None,[{'display_name': 'Affordable and clean energy...,[],[],[],74,"[https://openalex.org/W1553236419, https://ope...","[https://openalex.org/W620348747, https://open...",None,https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2024, 'cited_by_count': 47}, {'year'...",2024-12-06T09:25:31.940795,2016-06-24
1,https://openalex.org/W3037375387,https://doi.org/10.1016/j.envint.2020.105876,Short-term exposure to particulate matter (PM1...,Short-term exposure to particulate matter (PM1...,2020,2020-06-23,{'openalex': 'https://openalex.org/W3037375387...,en,"{'is_oa': True, 'landing_page_url': 'https://d...",review,journal-article,"[crossref, doaj, pubmed]","{'is_oa': True, 'oa_status': 'gold', 'oa_url':...","[{'author_position': 'first', 'author': {'id':...",[],1,5,[https://openalex.org/A5069680875],"[https://openalex.org/I128590013, https://open...","{'value': 3500, 'currency': 'USD', 'value_usd'...","{'value': 3500, 'currency': 'USD', 'value_usd'...",8.533,True,pdf,506,"{'value': 0.99778, 'is_in_top_1_percent': True...","{'min': 99, 'max': 100}","{'volume': '142', 'issue': None, 'first_page':...",False,False,"{'id': 'https://openalex.org/T10190', 'display...","[{'id': 'https://openalex.org/T10190', 'displa...",[{'id': 'https://openalex.org/keywords/nitroge...,"[{'id': 'https://openalex.org/C2780723490', 'w...","[{'descriptor_ui': 'D000393', 'descriptor_name...",4,"[{'is_oa': True, 'landing_page_url': 'https://...","{'is_oa': True, 'landing_page_url': 'https://d...",[{'display_name': 'Good health and well-being'...,[{'funder': 'https://openalex.org/F4320307856'...,[],[],84,"[https://openalex.org/W1525116145, https://ope...","[https://openalex.org/W4399338999, https://ope...","{'Air': [0], 'pollution': [1, 87, 155], 'is': ...",https://api.openalex.org/works?filter=cites:W3...,"[{'year': 2024, 'cited_by_count': 154}, {'year...",2024-12-03T13:35:34.799193,2020-07-02
2,https://openalex.org/W2032143374,https://doi.org/10.1016/s0023-6438(03)00088-4,Antimicrobial activity of essential oils on th...,Antimicrobial activity of essential oils on th...,2003,2003-11-01,{'openalex': 'https://openalex.org/W2032143374...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,journal-article,[crossref],"{'is_oa': False, 'oa_status': 'closed', 'oa_ur..."